In [4]:
%matplotlib inline
import pandas as pd
import seaborn as sns
import math
from tsmom_model import *
import numpy as np
from datetime import datetime
import statsmodels.api as sm

In [5]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [6]:
funding_liquidity, market_liquidity = liquidity_observables()

In [7]:
factors=get_all_factors()

In [8]:
results_df=pd.DataFrame()
for reg_fact in funding_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=funding_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(funding_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
#results_df=results_df.set_index(factors.columns)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [9]:
results_df.round(2).T

,0,1,2,3
TED Spread,-0.23,-1.17,-0.34,-0.76
TED SpreadTstats,-0.63,-2.69,-0.95,-0.95
LIBOR term repo,0.26,0.09,0.25,0.38
LIBOR term repoTstats,2.04,1.33,1.96,1.69


In [10]:
#results_df=pd.DataFrame()
for reg_fact in market_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=market_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
results_df=results_df.set_index(factors.columns)

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [13]:
results_df

,TED Spread,TED SpreadTstats,LIBOR term repo,LIBOR term repoTstats,PS,PSTstats,On versus off the run Treasuries,On versus off the run TreasuriesTstats
Amihud Annual,-0.227588,-0.628912,0.257041,2.038465,0.007031,0.542952,0.015909,0.753777
FHT Annual,-1.171923,-2.686925,0.089636,1.328174,0.006048,0.472648,-0.030785,-1.442293
Amihud Monthly,-0.336889,-0.952641,0.251962,1.959129,0.007705,0.576968,0.013985,0.587987
FHT Monthly,-0.755558,-0.947249,0.378527,1.686416,0.005011,0.344675,-0.031137,-1.561628


In [12]:
print results_df.T[[u'Amihud Annual', u'Amihud Monthly', u'FHT Annual', u'FHT Monthly']].round(2).to_latex()

\begin{tabular}{lrrrr}
\toprule
{} &  Amihud Annual &  Amihud Monthly &  FHT Annual &  FHT Monthly \\
\midrule
TED Spread                             &          -0.23 &           -0.34 &       -1.17 &        -0.76 \\
TED SpreadTstats                       &          -0.63 &           -0.95 &       -2.69 &        -0.95 \\
LIBOR term repo                        &           0.26 &            0.25 &        0.09 &         0.38 \\
LIBOR term repoTstats                  &           2.04 &            1.96 &        1.33 &         1.69 \\
PS                                     &           0.01 &            0.01 &        0.01 &         0.01 \\
PSTstats                               &           0.54 &            0.58 &        0.47 &         0.34 \\
On versus off the run Treasuries       &           0.02 &            0.01 &       -0.03 &        -0.03 \\
On versus off the run TreasuriesTstats &           0.75 &            0.59 &       -1.44 &        -1.56 \\
\bottomrule
\end{tabular}



In [ ]:
# Doing each leg of the observable direclty
#results_df=pd.DataFrame()
for reg_fact in market_liquidity.columns:
    results_param=[]
    results_t=[]
    for s in factors.columns:
        ind2=market_liquidity.diff()[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity.diff()[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        results_param.append(res.params[reg_fact])
        results_t.append(res.tvalues[reg_fact])
    results_df[reg_fact]=pd.Series(results_param)
    results_df[reg_fact+'Tstats']=pd.Series(results_t)
#results_df=results_df.set_index(factors.columns)#

In [14]:
ind2=market_liquidity.diff().dropna().index
ind=factors[s].loc[ind2].dropna().index
# Please make the fit robust 
res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity.diff()).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
#results_param.append(res.params[reg_fact])
#results_t.append(res.tvalues[reg_fact])

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  


In [15]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            FHT Monthly   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                     1.228
Date:                Mon, 30 Jul 2018   Prob (F-statistic):              0.295
Time:                        17:02:05   Log-Likelihood:                 529.04
No. Observations:                 204   AIC:                            -1052.
Df Residuals:                     201   BIC:                            -1042.
Df Model:                           2                                         
Covariance Type:                  HAC                                         
====================================================================================================
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                               -0.0003      0.001     -0.267      0.789      -0.003       0.002
PS                                   0.0053      0.014      0.365      0.715      -0.023       0.034
On versus off the run Treasuries    -0.0313      0.020     -1.567      0.117      -0.070       0.008
==============================================================================
Omnibus:                        1.581   Durbin-Watson:                   2.041
Prob(Omnibus):                  0.454   Jarque-Bera (JB):                1.471
Skew:                          -0.208   Prob(JB):                        0.479
Kurtosis:                       2.987   Cond. No.                         18.5
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 1 lags and without small sample correction
"""

# Showing the results of all factors and betas

In [80]:
all_macro=market_liquidity.join(funding_liquidity)

for s in factors.columns:

    result_list=[]
    t_list=[]

    for reg_fact in funding_liquidity.columns:
        ind2=funding_liquidity[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(funding_liquidity[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        result_list.append(res.params)
        t_list.append(res.tvalues)

    # All funding funding macro data
    ind2=funding_liquidity.dropna().index
    ind=factors[s].loc[ind2].dropna().index
    # Please make the fit robust 
    res=sm.OLS(factors[s].loc[ind],sm.add_constant(funding_liquidity).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    result_list.append(res.params)
    t_list.append(res.tvalues)

    for reg_fact in market_liquidity.columns:
        ind2=market_liquidity[reg_fact].dropna().index
        ind=factors[s].loc[ind2].dropna().index
        # Please make the fit robust 
        res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity[reg_fact]).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
        result_list.append(res.params)
        t_list.append(res.tvalues)

    # All market funding macro data
    ind2=market_liquidity.dropna().index
    ind=factors[s].loc[ind2].dropna().index
    # Please make the fit robust 
    res=sm.OLS(factors[s].loc[ind],sm.add_constant(market_liquidity).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    result_list.append(res.params)
    t_list.append(res.tvalues)

    # All Macro
    ind2=all_macro.dropna().index
    ind=factors[s].loc[ind2].dropna().index
    # Please make the fit robust 
    res=sm.OLS(factors[s].loc[ind],sm.add_constant(all_macro).loc[ind]).fit(cov_type='HAC',cov_kwds={'maxlags':1})
    result_list.append(res.params)
    t_list.append(res.tvalues)

    print s
    
    print pd.DataFrame(result_list)[[ u'TED Spread',u'LIBOR term repo',
                           u'PS', u'On versus off the run Treasuries']].T.round(2).to_latex().replace('NaN',' ')
    print pd.DataFrame(t_list)[[ u'TED Spread',u'LIBOR term repo',
                           u'PS', u'On versus off the run Treasuries']].T.round(2).to_latex().replace('NaN',' ')
    

/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:10: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  # Remove the CWD from sys.path while we load stuff.
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:26: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/Users/mrefermat/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:34: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation 

Amihud Annual
\begin{tabular}{lrrrrrrr}
\toprule
{} &     0 &     1 &     2 &    3 &    4 &    5 &     6 \\
\midrule
TED Spread                       & -0.04 &     &  0.30 &    &    &    &  0.30 \\
LIBOR term repo                  &     &  0.17 &  0.31 &    &    &    &  0.36 \\
PS                               &     &     &     &  0.0 &    &  0.0 & -0.03 \\
On versus off the run Treasuries &     &     &     &    & -0.0 & -0.0 & -0.00 \\
\bottomrule
\end{tabular}

\begin{tabular}{lrrrrrrr}
\toprule
{} &     0 &     1 &     2 &     3 &     4 &     5 &     6 \\
\midrule
TED Spread                       & -0.14 &     &  0.71 &     &     &     &  0.66 \\
LIBOR term repo                  &     &  0.97 &  1.08 &     &     &     &  1.12 \\
PS                               &     &     &     &  0.17 &     &  0.16 & -0.93 \\
On versus off the run Treasuries &     &     &     &     & -0.05 & -0.03 & -0.54 \\
\bottomrule
\end{tabular}

FHT Annual
\begin{tabular}{lrrrrrrr}
\toprule
{} &     0 &     

In [72]:
print pd.DataFrame(result_list)[[ u'TED Spread',u'LIBOR term repo',
                           u'PS', u'On versus off the run Treasuries']].T.round(2).to_latex()

\begin{tabular}{lrrrrrrr}
\toprule
{} &     0 &     1 &     2 &     3 &     4 &     5 &     6 \\
\midrule
TED Spread                       & -0.23 &   NaN &  0.23 &   NaN &   NaN &   NaN &  0.03 \\
LIBOR term repo                  &   NaN &  0.26 &  0.30 &   NaN &   NaN &   NaN &  0.22 \\
PS                               &   NaN &   NaN &   NaN &  0.01 &   NaN &  0.01 & -0.00 \\
On versus off the run Treasuries &   NaN &   NaN &   NaN &   NaN &  0.02 &  0.02 &  0.02 \\
\bottomrule
\end{tabular}



In [77]:
x= pd.DataFrame(t_list)[[ u'TED Spread',u'LIBOR term repo',
                           u'PS', u'On versus off the run Treasuries']].T.round(2).to_latex()

In [79]:
x

u'\\begin{tabular}{lrrrrrrr}\n\\toprule\n{} &     0 &     1 &     2 &    3 &     4 &     5 &     6 \\\\\n\\midrule\nTED Spread                       & -2.71 &     & -0.12 &    &     &     & -0.21 \\\\\nLIBOR term repo                  &     &  5.75 &  3.37 &    &     &     &  3.12 \\\\\nPS                               &     &     &     &  1.1 &     &  0.99 &  0.21 \\\\\nOn versus off the run Treasuries &     &     &     &    & -0.94 & -0.75 &  0.38 \\\\\n\\bottomrule\n\\end{tabular}\n'

In [30]:
results_param.append(res.params[reg_fact])
results_t.append(res.tvalues[reg_fact])

In [35]:
print res.summary()

                            OLS Regression Results                            
Dep. Variable:          Amihud Annual   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                    0.4178
Date:                Fri, 03 Aug 2018   Prob (F-statistic):              0.659
Time:                        12:13:36   Log-Likelihood:                 547.30
No. Observations:                 204   AIC:                            -1089.
Df Residuals:                     201   BIC:                            -1079.
Df Model:                           2                                         
Covariance Type:                  HAC                                         
                                       coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   